This contains the tools for publishing a public version of first reading resolutions.

This should be used after plenary. 

It will create a folder inside Google Drive labeled 'For campus feedback' with copies of all first reading resolutions

https://developers.google.com/docs/api/quickstart/python

In [ ]:
from googleapiclient.discovery import build

In [ ]:
import os
if os.path.exists('../token-1.json'):
    print('j')


In [ ]:

import API.CredentialsManager

In [ ]:
%cd ~/Dropbox/ResolutionManager
